<a href="https://colab.research.google.com/github/Sattwikmaiti/EcommerceAnalyis/blob/master/recommendationsystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import difflib #spelling mistakes from user. compare closest proximity to movies
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import csv
products = pd.read_csv('amazon_co-ecommerce_sample.csv')
products.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


In [ ]:
products.shape

(10000, 17)

In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      10000 non-null  object 
 1   product_name                                 10000 non-null  object 
 2   manufacturer                                 9993 non-null   object 
 3   price                                        8565 non-null   object 
 4   number_available_in_stock                    7500 non-null   object 
 5   number_of_reviews                            9982 non-null   object 
 6   number_of_answered_questions                 9235 non-null   float64
 7   average_review_rating                        9982 non-null   object 
 8   amazon_category_and_sub_category             9310 non-null   object 
 9   customers_who_bought_this_item_also_bought   8938 non-null   object 
 10 

In [ ]:
products.dropna(inplace=True)
products.isnull().sum()
products.shape

(511, 17)

In [ ]:
products.duplicated().sum()

0

In [ ]:
products.describe()

,number_of_answered_questions
count,511.000000
mean,1.994129
std,2.514656
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,28.000000


In [ ]:
ratings = products.iloc[:,7].values

collection=products.iloc[:,1].values

number_of_ratings=products.iloc[:,5].values

In [ ]:
import pandas as pd

# Sample data with ratings column
data = {'Ratings': ratings,'Collection Name':collection,'Number-of-Ratings':number_of_ratings}
df = pd.DataFrame(data)

# Extract only the rating from the 'ratings' column using string manipulation
df['Rating'] = df['Ratings'].str.split().str[0]

# Convert the 'rating' column to float data type
df['Rating'] = df['Rating'].astype(float)
df['Number-of-Ratings']=df['Number-of-Ratings'].astype(int)

# Drop the original 'ratings' column if needed
df.drop(columns=['Ratings'], inplace=True)

print(df)


                                       Collection Name  Number-of-Ratings  \
0                                Hornby 2014 Catalogue                 15   
1    CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...                 17   
2                     Hornby Santa's Express Train Set                 36   
3    Hornby Gauge Western Express Digital Train Set...                  1   
4    Hornby R3246TTS LNER 2-8-2 'Cock O' The North'...                  1   
..                                                 ...                ...   
506                            FIFA Scene It? DVD Game                  5   
507           DOMINO TILES RACKS. WOODEN. SET OF FOUR.                  7   
508           John Adams New Rummikub Junior Craft Kit                  1   
509                     Tobar 00435 Wooden Domino Race                 44   
510  Dominoes- double six, plastic,black spots,spin...                123   

     Rating  
0       4.9  
1       3.9  
2       4.3  
3       5.0  
4    

# **Popularity Basd Recommendation**

In [ ]:

df = df[df['Number-of-Ratings']>=10].sort_values('Rating',ascending=False).head(10)
df


,Collection Name,Number-of-Ratings,Rating
161,Churchill Mk.VII British Infantry Tank - 1:35 ...,15,5.0
203,Bruder 02121 Claas Rollant 250 Straw Baler,15,5.0
189,Bruder 03020 New Holland TG285 Tractor,22,5.0
14,DICEWORDS GAME,13,5.0
30,BRIO BRI-33061 Rail Cargo Harbour Set,14,5.0
52,Xcut set of 5 Scalloped Circle Nesting Die. Us...,10,5.0
0,Hornby 2014 Catalogue,15,4.9
331,Gel Pens - Brite Ideas (Assorted Pack of 30),37,4.9
342,Sorting Bowls,29,4.9
350,Star Wars X-Wing: Tantive IV Expansion Pack,19,4.9


# **Collaborative Based Recommendation**

In [ ]:
books = pd.read_csv('Books.csv')
users = pd.read_csv('Users.csv')
ratings = pd.read_csv('Ratings.csv')
ratings_with_name = ratings.merge(books,on='ISBN')
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df
avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating_df
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

ratings_with_name

<ipython-input-81-f3d7d60228db>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')
<ipython-input-81-f3d7d60228db>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...


In [ ]:

#users > 200 ratings and books with more than 50 ratings
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200

padhe_likhe_users = x[x].index


In [ ]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
final_ratings.drop_duplicates
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt.fillna(0,inplace=True)


In [ ]:
#each book is now a vector represented by 790 columns . so we need to find the similar vectors
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271448,271705,273979,274004,274061,274301,274308,275970,277427,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(pt)
similarity_scores.shape

(698, 698)

In [ ]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    #enumerate means to get index with similarity score  ={index,similarityscore}
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]

    data = []

    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))


        data.append(item)

    return data

In [ ]:
n=input("Enter the name of the product: ")
print("\nTop Recommended products are: \n")
recommend(n)


Enter the name of the product: Zoya

Top Recommended products are: 



[['Fine Things', 'Danielle Steel'],
 ['Secrets', 'DANIELLE STEEL'],
 ['Kaleidoscope', 'June Stepansky'],
 ['Exclusive', 'Sandra Brown']]

In [ ]:
tfv = TfidfVectorizer()
products['description'] = products['description'].fillna('')


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

vector = cv.fit_transform(products['description']).toarray()
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.        , 0.14242182, ..., 0.13333333, 0.16137431,
        0.        ],
       [0.        , 1.        , 0.35881405, ..., 0.        , 0.04279605,
        0.08119979],
       [0.14242182, 0.35881405, 1.        , ..., 0.03797915, 0.07814296,
        0.06977216],
       ...,
       [0.13333333, 0.        , 0.03797915, ..., 1.        , 0.12909944,
        0.        ],
       [0.16137431, 0.04279605, 0.07814296, ..., 0.12909944, 1.        ,
        0.        ],
       [0.        , 0.08119979, 0.06977216, ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
tfv_matrix = tfv.fit_transform(products['description'])
tfv_matrix.shape
print(tfv_matrix)

  (0, 4043)	0.18212590745314064
  (0, 1538)	0.10750718839791475
  (0, 986)	0.10701147838229667
  (0, 1198)	0.8273458121657047
  (0, 170)	0.36986413069705865
  (0, 2965)	0.3186111490906481
  (0, 1825)	0.10225715483143659
  (0, 4514)	0.10271681287927747
  (1, 3066)	0.0786508157811452
  (1, 3979)	0.07002878397754204
  (1, 811)	0.10711890258369193
  (1, 399)	0.125139798968902
  (1, 4824)	0.12198243257122153
  (1, 4711)	0.10711890258369193
  (1, 1847)	0.09153095764021456
  (1, 2408)	0.14517426233090422
  (1, 335)	0.13837198392339567
  (1, 6414)	0.125139798968902
  (1, 1373)	0.20831880898193578
  (1, 25)	0.15476153527556982
  (1, 4018)	0.07011169127660057
  (1, 3403)	0.11445255446475308
  (1, 3366)	0.14517426233090422
  (1, 3850)	0.08011386740295931
  (1, 3057)	0.041659424351655254
  :	:
  (509, 6454)	0.029840209412025805
  (509, 988)	0.11453938445943626
  (509, 5969)	0.06716651331587963
  (509, 1538)	0.06658557108987442
  (509, 986)	0.03313927425431202
  (509, 1825)	0.03166695713069631
  (5

In [ ]:
sig = cosine_similarity(tfv_matrix,tfv_matrix)
sig

array([[1.        , 0.        , 0.06604652, ..., 0.01269649, 0.01721023,
        0.        ],
       [0.        , 1.        , 0.25388215, ..., 0.0201472 , 0.05539892,
        0.04817409],
       [0.06604652, 0.25388215, 1.        , ..., 0.04959143, 0.07877549,
        0.04979897],
       ...,
       [0.01269649, 0.0201472 , 0.04959143, ..., 1.        , 0.10732558,
        0.01207041],
       [0.01721023, 0.05539892, 0.07877549, ..., 0.10732558, 1.        ,
        0.01377819],
       [0.        , 0.04817409, 0.04979897, ..., 0.01207041, 0.01377819,
        1.        ]])

In [ ]:
similarity

In [ ]:
indices = pd.Series(products.index,index=products['product_name']).drop_duplicates()
indices.head(20)

product_name
Hornby 2014 Catalogue                                                                                                              0
CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT ENGINE BOXED BOYS KIDS BATTERY                                                         2
Hornby Santa's Express Train Set                                                                                                   7
Hornby Gauge Western Express Digital Train Set with eLink and TTS Loco Train Set                                                   8
Hornby R3246TTS LNER 2-8-2 'Cock O' The North' P2 Class with TTS Sound                                                            30
Alps Glacier Express (Add-On 4-Car Set) (Model Train)                                                                             35
Hornby R1138 GWR Passenger Freight 00 Gauge Electric Train Set                                                                    47
20Pcs Single-Head Model Garden Lights Lamppost Lamp Layo

In [ ]:
def product_recommendation(title,sig=sig):
    indx = indices[title]

    #getting pairwise similarity scores
    sig_scores = list(enumerate(sig[indx]))

    # #sorting products
    # sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    #10 most similar products score
    sig_scores = sig_scores[1:11]

    #product indexes
    product_indices = [i[0] for i in sig_scores]

    #Top 10 most similar products
    return products['product_name'].iloc[product_indices]

In [ ]:
indices.head(100)

product_name
Hornby 2014 Catalogue                                                                  0
CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT ENGINE BOXED BOYS KIDS BATTERY             2
Hornby Santa's Express Train Set                                                       7
Hornby Gauge Western Express Digital Train Set with eLink and TTS Loco Train Set       8
Hornby R3246TTS LNER 2-8-2 'Cock O' The North' P2 Class with TTS Sound                30
                                                                                    ... 
CARRERA RC 370162052 Turnator 2.4GHz (RTR)                                          2119
K'Nex Princess Peach Birthday Girl Building Set                                     2153
Carrera Go Red Bull F1 Champions Racing Set                                         2155
Thunder Tiger Pull Start PN0258 For Pro21/28BX-R                                    2287
HPI E10 Drift RTR with Vaughn Gittin Jr. Monster Ford Mustang Body 111664           2301
Length: 

In [ ]:
n=input("Enter the name of the product: ")
print("\nTop Recommended products are: \n")
print(product_recommendation(n))

Enter the name of the product: Hornby 2014 Catalogue

Top Recommended products are: 

2      CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...
7                       Hornby Santa's Express Train Set
8      Hornby Gauge Western Express Digital Train Set...
30     Hornby R3246TTS LNER 2-8-2 'Cock O' The North'...
35     Alps Glacier Express (Add-On 4-Car Set) (Model...
47     Hornby R1138 GWR Passenger Freight 00 Gauge El...
87     20Pcs Single-Head Model Garden Lights Lamppost...
126    Fisher Price Thomas and Friends Take-n-play(TM...
164    Hornby R1167 Flying Scotsman 00 Gauge Electric...
195                    Mehano - Coffret TGV duplex 3.5 m
Name: product_name, dtype: object
